# Produit mixe non linéaire
# Prise en compte de la demande client

Je vais recréer le même NLP que dans le fichier Excel ci joint, et voir si le solveur
GEKKO trouve bien le même résultat !


Code by Estelle Derrien 

* L'histoire
- On a un magasin de jouets qui fabrique 5 jouets.

- Le service commercial nous a communiqué les équations linéaires de la demande de chacun des chaque objets, il s'agit de données prévisionnelles de vente.
- On a aussi le service production qui nous a communiqué ces contraintes de temps de travail MAXIMUM.
- Combien de jouets faut il produire afin de maximiser le profit de notre entreprise ?


Source: 
Bernard Taylor - Introduction to management science - Non linear optimizations



In [27]:
# On importe Gekko
from gekko import GEKKO

m = GEKKO(remote=False) # Initialize gekko

#The example problem that you referenced uses the default IPOPT solver. To get a binary or integer solution, switch to the APOPT solver.
m.options.SOLVER = 1


In [28]:
# On initialise nos variables de temps de travail max exprimées en heures
temps_assemblage = 800
temps_inspection = 500

# On initialise nos variables de coût de production variable pour chaque jouet.
cout_robot = 60
cout_auto = 45
cout_camion = 38
cout_poupee = 40 
cout_deguisement = 65

# On crée les variables de décision de type entière puisqu'on ne peut pas avoir un demi jouet, c'est impossible.
# Ces variables contiendront le nombre de jouets à réaliser, ca va être calculé par le solveur à la fin.
robot = m.Var(value=1,lb=0,ub=10000,integer=True)
auto = m.Var(value=1,lb=0,ub=10000,integer=True)
camion = m.Var(value=1,lb=0,ub=10000,integer=True)
poupee = m.Var(value=1,lb=0,ub=10000,integer=True)
deguisement = m.Var(value=1,lb=0,ub=10000,integer=True)


# On entre les Equations de contrainte de temps de travail, exprimées en heures.
m.Equation(2 * robot + 3 * auto + 2 * camion + 3 * poupee + 1 * deguisement <= temps_assemblage)
m.Equation(2 * robot + 1 * auto + 2 * camion + 1 * poupee + 2 * deguisement <= temps_inspection)


# On entre les Equations de la demande qui ont été communiquées par le service commercial.
# On les entre là, pour que la fonction objectif soies moins compliquée à lire après.
demande_robot = m.Var()
demande_auto = m.Var()
demande_camion = m.Var()
demande_poupee = m.Var()
demande_deguisement = m.Var()
m.Equation(demande_robot == (220 - 0.4 * robot)- cout_robot)
m.Equation(demande_auto == (180 - 0.2 * auto) - cout_auto)
m.Equation(demande_camion == (230 - 0.2 * camion) - cout_camion)
m.Equation(demande_poupee == (175 - 0.2 * poupee) - cout_poupee)
m.Equation(demande_deguisement == (220 - 0.2 * deguisement) - cout_deguisement)

# Voici notre fonction objectif , qui tient compte de la demande
m.Maximize( (demande_robot * robot) + (demande_auto * auto) + (demande_camion * camion) + (demande_poupee * poupee) + (demande_deguisement * deguisement)  ) 







In [29]:
# On imprime le résultat 
m.options.IMODE = 3 # Steady state optimization

m.solve(disp=False) # Solve

print('Solution')
print('robot: ' + str(robot.value))
print('auto: ' + str(auto.value))
print('camion: ' + str(camion.value))
print('poupee: ' + str(poupee.value))
print('deguisement: ' + str(deguisement.value))
print('Objectif: ' + str(m.options.objfcnval))

Solution
robot: [13.0]
auto: [86.0]
camion: [106.0]
poupee: [86.0]
deguisement: [45.0]
Objectif: -46948.8


Le résultat est globalement le même que dans le fichier Excel ci joint, les différences minimes sont dues au fait que les variables de décisions sont définies en continue sous excel et en entiers sous gekko ( En entier, c'est ce qu'il faut.)

Le solveur Gekko, confirme donc une fois de plus, qu'il peut traiter des problèmes identiques à ceux
d'excel , et qu'il trouve le même résultat, en non linéaire, que le solveur GRG de excel.
